# 디자인 패턴  
함수나 메소드에 대한 디자인 패턴에는 여러가지가 있지만 이번 장에서 소개하는 두가지 방법이 가장 대표적인 형태라고 할 수 있다.  
먼저 이번 장에서 사용하는 용어에 대해 알아보자.  

- 콘텍스트(Context)
콘텍스트는 전략을 호출하는 일종의 클라이언트라고 생각하면 된다.  
여러 알고리즘 중 하나를 선택하여 적용할 수 있도록 한다.

- 전략  
고전적인 방식에서는 부모 클래스와 자식 클래스로 구현한다. 부모 클래스는 공통된 인터페이스를 담당하고, 자식 클래스는 구체적인 알고리즘이 구현되어 있다.

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

# Context
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

In [2]:
class Promotion(ABC): # 전략, 추상 클래스
    # 부모 클래스로, 자식 클래스가 구현해야 하는 메소드들을 정의한다.
    @abstractmethod
    def discount(self, order):
        """ 할인액 반환 """
    
class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct = {item.product for item in order.cart}
        if len(distinct) >= 10:
            return order.total() * .07
        return 0

In [3]:
# 함수 지향형으로 바꾼 경우
# 위에서는 부모 클래스를 상속 받아 추상 메소드를 구현하는 식으로 진행했다.
# 파이썬에서는 이를 함수만으로도 활용할 수 있기 때문에 아래와 같은 구현이 가능해진다.

def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    distinct = {item.product for item in order.cart}
    if len(distinct) >= 10:
        return order.total() * .07
    return 0